In [ ]:
import pyaudio
import matplotlib.pyplot as plt
import numpy as np
import scipy.io.wavfile as wav
%matplotlib inline


FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
CHUNK = 1024
RECORD_SECONDS = 5

def recording():
    audio = pyaudio.PyAudio()

    # start Recording
    stream = audio.open(format=FORMAT,
                        channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK)
    frames = []
    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)

    stream.stop_stream()
    stream.close()
    audio.terminate()
    return frames

In [ ]:
# recording first samples
amplitude1 = np.fromstring(b''.join(recording()), np.int16)

In [ ]:
# recording second samples
amplitude2 = np.fromstring(b''.join(recording()), np.int16)

In [ ]:
plt.xlim(len(amplitude1))
plt.plot(amplitude1, 'b', label='first sample', alpha=0.5)
plt.plot(amplitude2, 'r', label='second sample', alpha=0.5)
plt.legend()

In [ ]:
from IPython.display import display, Audio

plt.scatter(amplitude1, amplitude2, s=1, marker='x', alpha=0.2)
display(Audio(amplitude1, rate=RATE))
display(Audio(amplitude2, rate=RATE))

In [ ]:
# Step 2:  get two random remixes

# Randomly mix
r = np.random.rand(4).reshape(2, 2)
print('random matrix: ')
print(r)

x1, x2 = np.dot(r, (amplitude1, amplitude2))

#x1 = x1 - mean(x1)
#x2 = x2 - mean(x2)

plt.subplot(211)
plt.xlim(len(x1))
plt.title('mixed (x1)')
plt.plot(x1)
plt.subplot(212)
plt.title('mixed (x2)')
plt.xlim(len(x2))
plt.plot(x2)
plt.tight_layout()

plt.figure()
plt.scatter(x1, x2, s=1, marker='x', alpha=0.2)

display(Audio(x1, rate=RATE))
display(Audio(x2, rate=RATE))

#wav.write('ics_mixsig1.wav',RATE,x1)
#wav.write('ica_mixsig2.wav',RATE,x2)

In [ ]:
sample1 = x1[::10]
sample2 = x2[::10]
def kurtosis_of_mixture(c1):
    c2 = np.sqrt(1 - c1 ** 2)
    s = c1 * sample1 + c2 * sample2
    s = s / np.std(s)
    k = mean([item ** 4 for item in s]) - 3
    return k

c_array = np.arange(-1,1,0.001)
k_array = [kurtosis_of_mixture(item) for item in c_array]

plt.plot(c_array, k_array)

In [ ]:
# Step 4:  Find extrema, plot signals that come out.

index1 = k_array.index(max(k_array))
c1 = c_array[index1]
c2 = np.sqrt(1 - c1 ** 2)
s  = np.array([int16(item) for item in c1 * x1 + c2 * x2])
#wav.write('ica_remixmix1.wav',RATE,s)
print(index1, c1, c2)

index2 = k_array.index(min(k_array))
c1 = c_array[index2]
c2 = np.sqrt(1 - c1 ** 2)
s2  = np.array([int16(item) for item in c1 * x1 + c2 * x2])
#wav.write('ica_remixmix2.wav',RATE,s2)
print(index1, c1, c2)

display(Audio(s, rate=RATE))
display(Audio(s2, rate=RATE))